In [1]:
#dataset link: https://drive.google.com/drive/folders/1n67sVTTzye4jtLfk8n-sa2fH2gTx5Ywt?usp=share_link

In [2]:
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
DATA_DIR = "../Dataset Korosi/"

2023-01-01 23:54:24.160011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 23:54:24.288993: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-01 23:54:24.896632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-01 23:54:24.896684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
    )

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    
)

train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=32,class_mode='categorical',target_size=(224,224))   
val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, subset='validation', batch_size=32,class_mode='categorical',target_size=(224,224))   

Found 304 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [4]:
## resep yang nemu di: https://www.kaggle.com/code/vortexkol/alexnet-cnn-architecture-on-tensorflow-beginner

# model_alexnet = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2)),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(3,3)),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2)),

#     tf.keras.layers.Flatten(),

#     tf.keras.layers.Dense(1024,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1024,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(3,activation='softmax')  
# ])

## resep yang nemu di: https://medium.com/swlh/alexnet-with-tensorflow-46f366559ce8

model_alexnet = tf.keras.models.Sequential()
model_alexnet.add(tf.keras.layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=(224,224,3)))
model_alexnet.add(tf.keras.layers.Conv2D(96, 11, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.MaxPooling2D(3, strides=2))
model_alexnet.add(tf.keras.layers.Conv2D(256, 5, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.MaxPooling2D(3, strides=2))
model_alexnet.add(tf.keras.layers.Conv2D(384, 3, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.Conv2D(384, 3, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.Conv2D(256, 3, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.Flatten())
model_alexnet.add(tf.keras.layers.Dense(4096, activation='relu'))
model_alexnet.add(tf.keras.layers.Dropout(0.5))
model_alexnet.add(tf.keras.layers.Dense(4096, activation='relu'))
model_alexnet.add(tf.keras.layers.Dropout(0.5))
model_alexnet.add(tf.keras.layers.Dense(3, activation='softmax'))


2023-01-01 23:54:25.736492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-01 23:54:25.765491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-01 23:54:25.765668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-01 23:54:25.766092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
model_alexnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 56, 56, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 56, 56, 96)        0         
                                                                 
 activation (Activation)     (None, 56, 56, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 256)         614656    
                                                        

In [6]:
import matplotlib.pyplot as plt

In [7]:
f = open("alexnet.csv", "w")
f.write("epochs,loss,val_loss,acc,val_acc")
f.close()


In [8]:
Wsave = model_alexnet.get_weights()
for times in range(12):
    epo = (times+1)*10
    model_alexnet.set_weights(Wsave)
    model_alexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = model_alexnet.fit(train_generator,validation_data=val_generator,epochs=epo, verbose = 2)

    f = open("alexnet.csv", "a")
    f.write("\n"+str(epo)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./alexnet/loss_'+str(epo)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./alexnet/acc_'+str(epo)+'.png')
    
    plt.clf()

Epoch 1/10


2023-01-01 23:54:32.846844: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8300
2023-01-01 23:54:33.446680: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


2/2 - 9s - loss: 1.0981 - acc: 0.4500 - val_loss: 1.0969 - val_acc: 0.4833 - 9s/epoch - 4s/step
Epoch 2/10
2/2 - 5s - loss: 1.0971 - acc: 0.4833 - val_loss: 1.0945 - val_acc: 0.4833 - 5s/epoch - 3s/step
Epoch 3/10
2/2 - 6s - loss: 1.0941 - acc: 0.4833 - val_loss: 1.0915 - val_acc: 0.4833 - 6s/epoch - 3s/step
Epoch 4/10
2/2 - 6s - loss: 1.0912 - acc: 0.5167 - val_loss: 1.0879 - val_acc: 0.5167 - 6s/epoch - 3s/step
Epoch 5/10
2/2 - 6s - loss: 1.0879 - acc: 0.5500 - val_loss: 1.0822 - val_acc: 0.7000 - 6s/epoch - 3s/step
Epoch 6/10
2/2 - 6s - loss: 1.0801 - acc: 0.6833 - val_loss: 1.0776 - val_acc: 0.7333 - 6s/epoch - 3s/step
Epoch 7/10
2/2 - 5s - loss: 1.0794 - acc: 0.6667 - val_loss: 1.0698 - val_acc: 0.7000 - 5s/epoch - 3s/step
Epoch 8/10
2/2 - 6s - loss: 1.0689 - acc: 0.6500 - val_loss: 1.0630 - val_acc: 0.7167 - 6s/epoch - 3s/step
Epoch 9/10
2/2 - 6s - loss: 1.0658 - acc: 0.6333 - val_loss: 1.0553 - val_acc: 0.4833 - 6s/epoch - 3s/step
Epoch 10/10
2/2 - 6s - loss: 1.0544 - acc: 0.533

<Figure size 640x480 with 0 Axes>